In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import data_utils
import numpy as np

In [2]:
class DrummerDataGenerator():

    def __init__(self, data, controller, x_key, y_key, rng_start, rng_end, batch_size, win_size, normalize=True):
        self.x_data = np.asarray(data[controller][x_key][rng_start:rng_end])
        self.y_data = np.asarray(data[controller][y_key][rng_start:rng_end])
        self.x_key = x_key
        self.y_key = y_key
        self.controller = controller
        self.batch_size = batch_size
        self.win_size = win_size

    # def midi_to_label(self, midi):


    def choose_random_example(self, win_size):
        rnd_idx = np.random.randint(0, self.x_data.shape[0] - win_size)
        x = self.x_data[rnd_idx : rnd_idx + win_size]
        y = self.y_data[rnd_idx : rnd_idx + win_size]
        return x, y

    def generate(self):
        x_iter = iter(self.x_data)
        y_iter = iter(self.y_data)

        while True:
            x_batch = []
            y_batch = []

            for _ in range(self.batch_size):
                x, y = self.choose_random_example(self.win_size)
                x_batch.append(x)
                y_batch.append(y)

            yield np.asarray(x_batch), np.asarray(y_batch)


In [6]:
data = data_utils.load_json("../CapturedMotion/2021-04-09-084809/capture_2021-04-09-084809.json")
batch_size = 32
dataGen = DrummerDataGenerator(data, 
                                controller="c0",
                                x_key="midi_note",
                                y_key="ang_vel",
                                rng_start=1400,
                                rng_end=3300,
                                batch_size=batch_size,
                                win_size=64)
generator = dataGen.generate()

In [1]:
x_b, y_b = next(generator)
print(x_b.shape, y_b.shape)
plt.subplot(121)
plt.title(dataGen.x_key)
plt.plot(x_b[0])
plt.subplot(122)
plt.title(dataGen.y_key)
plt.plot(y_b[0])
plt.show()


NameError: name 'generator' is not defined

In [8]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, LSTM, Dense, Input, MaxPooling1D, UpSampling1D

In [9]:
kernel_size = 7

model = Sequential()

model.add(Input(shape=(x_b.shape[1], x_b.shape[2])))
model.add(Conv1D(32, kernel_size, padding='same'))
model.add(MaxPooling1D())
model.add(Conv1D(32, kernel_size, padding='same'))
model.add(MaxPooling1D())
model.add(Conv1D(64, kernel_size, padding='same'))
model.add(MaxPooling1D())
model.add(LSTM(32, return_sequences=True))
model.add(UpSampling1D(size=2))
model.add(Conv1DTranspose(32, kernel_size, padding="same"))
model.add(UpSampling1D(size=2))
model.add(Conv1DTranspose(16, kernel_size, padding="same"))
model.add(UpSampling1D(size=2))
model.add(Conv1DTranspose(8, kernel_size, padding="same"))

model.add(Conv1D(y_b.shape[2], kernel_size, padding='same'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 64, 32)            480       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 32, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 32)            7200      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 64)            14400     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 8, 32)             1

In [10]:
model.compile(optimizer="adam",
            loss=tf.keras.losses.MSE,
            metrics=["MSE"])
model.fit(generator, 
            batch_size=batch_size,
            epochs=10, 
            steps_per_epoch=500)

Epoch 1/10
500/500 [==============================] - 22s 23ms/step - loss: 2.1895 - MSE: 2.1895
Epoch 2/10
500/500 [==============================] - 12s 23ms/step - loss: 1.6432 - MSE: 1.6432
Epoch 3/10
500/500 [==============================] - 13s 27ms/step - loss: 1.4594 - MSE: 1.4594
Epoch 4/10
500/500 [==============================] - 12s 25ms/step - loss: 1.3789 - MSE: 1.3789
Epoch 5/10
300/500 [=================>............] - ETA: 5s - loss: 1.3260 - MSE: 1.3260

KeyboardInterrupt: 